In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import os
import traceback
import time
import random

In [2]:
KOSPI200 = pd.read_csv("KOSPI200.csv", encoding='cp949')

In [3]:
code = []
for i in range(0, 200):
    a = str(KOSPI200["종목코드"][i])
    code.append("0"*(6-len(a)) + a)
    

In [4]:
KOSPI200["code"] = code

In [5]:
KOSPI200.head()

,종목코드,종목,code
0,5930,삼성전자,005930
1,660,SK하이닉스,000660
2,207940,삼성바이오로직스,207940
3,35420,NAVER,035420
4,68270,셀트리온,068270


In [6]:
url = "https://finance.naver.com/item/sise_day.nhn?code=035420"

In [9]:
def get_last_page(code):
    url = "https://finance.naver.com/item/sise_day.nhn?code=035420"
    params = {"code":code}
    res = requests.get(url, params = params)
    res.encoding = 'utf-8'
    soap = BeautifulSoup(res.text, 'lxml')
    
    # 마지막 페이지 숫자 가져오기
    el_table_navi = soap.find("table", class_="Nnavi")
    el_td_last = el_table_navi.find("td", class_="pgRR")
    pg_last = el_td_last.a.get('href').rsplit('&')[1]
    pg_last = pg_last.split('=')[1]
    pg_last = int(pg_last)

    return pg_last

In [10]:
print(get_last_page(KOSPI200["code"][0]))

5930


In [11]:
#페이지 별로 데이터 가져오기
def parse_page(code, page):
    try:
        url = 'http://finance.naver.com/item/sise_day.nhn?code={code}&page={page}'.format(code=code, page=page)
        res = requests.get(url)
        time.sleep(random.uniform(1,3))
        _soap = BeautifulSoup(res.text, 'lxml')
        _df = pd.read_html(str(_soap.find("table")), header=0)[0]
        _df = _df.dropna()
        return _df
    except Exception as e:
        traceback.print_exc()
    return None


In [ ]:
for i in range(2,10):
    code = KOSPI200["code"][i]

    pg_last = get_last_page(code)

    #시작과 끝 날짜 설정
    str_datefrom = datetime.datetime.strftime(datetime.datetime(year=2015, month=1, day=1), '%Y.%m.%d')
    str_dateto = datetime.datetime.strftime(datetime.datetime.today(), "%Y.%m.%d")
    
    
    df = None
    for page in range(1, pg_last + 1):
        _df = parse_page(code, page)
        _df_filtered = _df[_df["날짜"] > str_datefrom]
        if df is None:
            df = _df_filtered
        else:
            df = pd.concat([df, _df_filtered])
        if len(_df) > len(_df_filtered):
            break

    
    
    path = os.path.join(os.getcwd()+"\\stock_value","{}_{}_{}_{}.csv".format(code, KOSPI200["종목"][i], str_datefrom, str_dateto) )
    df.to_csv(path, encoding='utf-8-sig')
    
    print("{} DONE".format(KOSPI200["종목"][i]))